In [ ]:
import warnings

from data_loading import *
from utils import DateIter
from threshold_edge_detection import *
from IPython.display import clear_output

In [ ]:
parent_dir = 'A:/Gits/Ionosphere/raw_data/multisource_data_NA_20m_T0_24hr/'

full_xarr = create_xarr(
    parent_dir=parent_dir,
    expected_shape=(720, 300),
    dtype=(np.uint16, np.float32),
    apply_fn=mad,
    plot=False,
)

label_df = create_label_df(
    csv_path='../ionospheric_disturbance_segmentation/labels/official_labels.csv'
)

In [ ]:
data_out_path = 'processed_data/full_data.joblib'
label_out_path = 'labels/labels.joblib'

print(full_xarr.indexes['date'], label_df.index)
joblib.dump(full_xarr, data_out_path)
joblib.dump(label_df, label_out_path)

# %pip install netCDF4
# full_xarr.to_netcdf('full_data.nc')

In [ ]:
data_out_path = 'processed_data/full_data.joblib'
label_out_path = 'labels/labels.joblib'

date_iter = DateIter(data_out_path) #, label_df=label_out_path)

In [ ]:
def save_wrap(save_dir, fmt='%Y-%m-%d', ext='.png', **kwargs):
    os.makedirs(save_dir, exist_ok=True)
    def wrapped(date):
        date_str = pd.to_datetime(date).strftime(fmt)
        file_path = os.path.join(save_dir, date_str + ext)
        plt.savefig(file_path, **kwargs)
        return
    return wrapped

In [ ]:
def run_edge_detect(
    dates,
    x_trim=1/12,
    y_trim=.08,
    sigma=4.2, # 3.8 was good
    qs=[.4, .5, .6],
    occurence_n = 60,
    i_max=30,
    plot=True,
    clear_every=100,
    plt_save_path=None,
    csv_save_path=None,
    display_head=None,
    thresh=None,
):
    sums, processed_dates = list(), list()
    if plt_save_path is not None:
        save_plt = save_wrap(plt_save_path)
    else:
        save_plt = None
        
    final_edge_dict = dict()
    
    if dates == 'all':
        date_gen = date_iter.iter_all()
    else:
#         try:
        date_gen = date_iter.iter_dates(dates, raise_missing=False)
#         except KeyError as ke:
#             missing_dates = 
            
    
    print(next(date_gen))
    for i, (date, arr) in enumerate(date_gen): #('2018-01-01','2017-01-30'))):
        if arr is None:
            warnings.warn(f'Date {date} has no input')
            continue
            
        if not i % clear_every:
            clear_output()

        xl_trim, xr_trim = x_trim if isinstance(x_trim, (tuple, list)) else (x_trim, x_trim)
        yl_trim, yr_trim = x_trim if isinstance(y_trim, (tuple, list)) else (y_trim, y_trim)
        xr, xl = math.floor(xl_trim * arr.shape[0]), math.floor(xr_trim * arr.shape[0])
        yr, yl = math.floor(yl_trim * arr.shape[1]), math.floor(yr_trim * arr.shape[1])

        arr = arr[xr:-xl, yr:-yl]
#         display(
#             arr.coords,
#             arr.dims,
#         )
#         assert False
        heights = arr.coords['height']
        times = arr.coords['time'].to_pandas().dt.strftime('%H:%M')
        
        arr = np.nan_to_num(arr, nan=0)

        arr = gaussian_filter(arr.T[::-1,:], sigma=(sigma, sigma)) 
        med_lines, min_line, minz_line = measure_thresholds(
            arr[::-1,:], 
            qs=qs, 
            occurrence_n=occurence_n, 
            i_max=i_max
        )

        data = pd.DataFrame(np.array(med_lines).T, index=times, columns=qs).reset_index(names='Time')
        if thresh is None:
            min_line = pd.DataFrame(min_line, index=times, columns=['Height']).reset_index(names='Time')
        elif isinstance(thresh, dict):
            min_line = data[[thresh[date]]]
        elif isinstance(thresh, float):
            min_line = data[[thresh]]
        else:
            raise ValueError(f'Threshold {thresh} of type {type(thresh)} is invalid')
#         data_uncertainty = data.rolling(32).corr().groupby(level=0).apply(non_diag_corr)

        print(min_line.shape)
        final_edge_dict[date] = min_line.squeeze()

        if plot or save_plt is not None:
            fig, ax = plt.subplots(1, 1, figsize=(15,8))
            plt.title(f'| {date} |')
#             ax.axis('off')
            sns.heatmap(
                pd.DataFrame(arr[::-1], index=heights, columns=times), 
                robust=True, 
                cbar=False, 
                ax=ax,
            )
            ax.invert_yaxis()

            sns.lineplot(data=data, alpha=.75, dashes=False, ax=ax, palette='light:b')
            sns.lineplot(data=min_line, x='Time', y='Height', color='white', alpha=1, dashes=False, ax=ax)

            if save_plt is not None:
                save_plt(date)
            
#             sums.append(round(s, 0))
            processed_dates.append(date)
            
            if plot:
                plt.show()
            else:
                plt.clear()
                
    ################ Special Request, should be removed ######################################################
#     pd.Series(sums, index=dates, name='Total Counts').to_frame().to_csv('./January2020Edges/total_counts.csv')
    ##########################################################################################################
    
    final_edge_df = pd.DataFrame(final_edge_dict)
    if csv_save_path:
        final_edge_df.to_csv(csv_save_path)
    
    if display_head:
        display(final_edge_df.head(display_head))
    
    return final_edge_df

In [ ]:
thresh_df = pd.read_excel('Edges_2020_MLW.xlsx', sheet_name='Sheet2')
thresh_dict = thresh_df.set_index('Date').squeeze().to_dict()
thresh_dict

In [ ]:
# run_edge_detect(
#     [('2022-10-27','2022-10-31')], 
#     csv_save_path=None,
#     plot=True,
#     plt_save_path='A:/Gits/Ionosphere/ionospheric_edge_detection/outputs/October2022Edges'
# ).head()

In [ ]:
today = pd.to_datetime('today').strftime('%Y-%m-%d')
full_edge_path = f'processed_data/final_edges_{today}_run.csv'

final_edge_df = run_edge_detect(
#     'all',
    sorted(thresh_dict.keys()),
    qs=[.4,.5,.6],
    clear_every=10,
    csv_save_path=full_edge_path,
    plot=False,
    thresh=thresh_dict,
)


In [ ]:
def plot_date(date_iter, edge_df, date, side_trim=.08):
    if isinstance(edge_df, str):
        edge_df = pd.read_csv(edge_df, index_col=0)
        edge_df.columns = pd.to_datetime(edge_df.columns)
    
    date = pd.to_datetime(date)
    arr = date_iter.get_date(date)
    x_trim = math.floor(side_trim * arr.shape[0])
    y_trim = math.floor(side_trim * arr.shape[1])
    arr = arr[x_trim:-x_trim, y_trim:-y_trim]
    
    edge = edge_df[date]
    
    fig, ax = plt.subplots(1, 1, figsize=(15,8))
    plt.title(date)
    ax.axis('off')
    sns.heatmap(arr.T, robust=True, cbar=False, ax=ax)
    ax.invert_yaxis()

    sns.lineplot(data=edge, color='white', alpha=1, dashes=False, ax=ax)    
    plt.show()
    return

plot_date(date_iter, final_edge_df, '2020-01-20', side_trim=side_trim)